## [ Combining and Merging Datasets ]
Data contained in pandas objects can be combined in a number of ways:

#### Table:

| Method         | Use Case                                     | Join Type       |
|----------------|----------------------------------------------|------------------|
| `concat()`     | Stack DataFrames (vertically/horizontally)   | Union of data    |
| `merge()`      | Join on common columns                       | Inner/Outer/Left/Right |
| `join()`       | Join based on index                          | Simpler syntax   |
| `combine_first()` | Fill missing values from another df       | Null-filler      |
| `update()`     | Overwrite values with another df             | In-place         |


####  When to use what?

| Situation                                        | Use this          |
|-------------------------------------------------|-------------------|
| Stacking datasets with same columns             | `concat()`        |
| Combining tables using key columns (like SQL)   | `merge()`         |
| Joining on index (instead of column)            | `join()`          |
| Filling missing values from another dataset     | `combine_first()` |
| Updating values in-place                        | `update()`        |


In [1]:
import numpy as np 
import pandas as pd 

## [ Database-Style DataFrame Joins ]
- `merge` or `join` operations combine datasets by linking rows using one or more `keys`.
- These operations are particularly important in relational databases (e.g., SQL-based)
- `pandas.merge` function is the main entry point for using these algorithms on our data

In [2]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   d      2


this is an example of a many-to-one join

In [3]:
# this is an example of a many-to-one join
# the data in df1 has multiple rows labeled a and b, whereas df2 has only one row for each value in the key column

# calling pd.merge() with these objects
pd.merge(df1, df2)

# for each matching key, it pairs every row from df1 with every matching row from df2. This is called a Cartesian Product

# if not specified which column to join on, pd.merge uses the overlapping column names as the keys

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [4]:
# specifying explicitly
pd.merge(df1, df2, on="key", sort="key")
# means join both dataframes where df1.key == df2.key. Only keep the matches

# joins on common values of key
# each matching key does a Cartesian product of the matching rows
    # here: 'a' matches → 3 rows (df1) × 1 row (df2) = 3 rows
    #       'b' matches → 3 rows (df1) × 1 row (df2) = 3 rows
    # total: 6 rows

# non-matching keys are dropped by default

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1


In [5]:
# merge tow DataFrames with different column names for the keys
df3 = pd.DataFrame({
    "lkey": ["b", "b", "a", "c", "a", "a", "b"],
    "data1": pd.Series(range(7), dtype="Int64")})

df4 = pd.DataFrame({
    "rkey": ["a", "b", "d"],
    "data2": pd.Series(range(3), dtype="Int64")})

print(df3)
print(df4)

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6
  rkey  data2
0    a      0
1    b      1
2    d      2


In [6]:
# merge operation
pd.merge(df3, df4, left_on="lkey", right_on="rkey", sort="lkey")

# means:
#     use df3["lkey"] as the join key from the left DataFrame.
#     use df4["rkey"] as the join key from the right DataFrame.

# it is equivalent to saying: join rows in df3 and df4 where the value in lkey(from df3) equals the value in rkey(from df4)

# Only keys a and b match → rows with lkey = a or b in df3 will be included.
# For each match, pandas does a Cartesian product of matching rows.


# Summary:
    # left_on and right_on are used when column names differ in each DataFrame.
    # Matching is done just like before (inner join by default).
    # Useful for merging datasets with different naming conventions.

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1


- Four types of Joins in pandas.merge
- Each join type controls which keys (values in the joining columns) appear in the result

| Join Type | Keeps Keys From | Fills with NaN |
|-----------|-----------------|----------------|
| inner     | both            | no             |
| left      | left            | right columns  |
| right     | right           | left columns   |
| outer     | all             | both sides     |


In [7]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


In [8]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


many-to-many merges form the cartesian product of the matching keys

In [9]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})
print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5
  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4


In [10]:
pd.merge(df1, df2, on="key", how="left", sort="key")

,key,data1,data2
0,a,2,0
1,a,2,2
2,a,4,0
3,a,4,2
4,b,0,1
5,b,0,3
6,b,1,1
7,b,1,3
8,b,5,1
9,b,5,3


In [11]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


to merge with multiple keys, pass a list of column names

In [12]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})

right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})

print(left)
print(right)

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


In [13]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

# An outer join means:
#     Give me all the combinations of key1 + key2 from both tables, even if there is no match
# So it includes:
#     All rows from left
#     All rows from right
#     If a match is found on both key1 and key2, join the data
#     If not found, fill missing part with NaN

# All unique (key1, key2) combinations across both:
    # key1        key2
    # foo         one
    # foo         two
    # bar         one
    # bar         two

# look at the matches and build final table

# example: match for foo + one
#   from left: lval = 1
#   from right: two matches : rval = 4 and 5
#   this gives two rows
    # key1    key2    lval    rval
    # foo     one     1       4
    # foo     one     1       5

,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [14]:
# treatment of overlapping column names
pd.merge(left, right, on="key1")

# we're merging only on key1
# other columns (key2, lval, rval) ar overlapping or unreleated
# so pandas will include both key2 columns: one from each dataFrame
# since both left and right have key2, pandas adds suffixes to avoid name clashes

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [15]:
# pandas.merge has a suffixes option for specifying strings to append to overlapping names in the left and right DataFrame objects
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7



#### Basic Arguments:

| Argument         | Description                                                                 |
|------------------|-----------------------------------------------------------------------------|
| `left`           | First DataFrame to merge                                                    |
| `right`          | Second DataFrame to merge                                                   |
| `how`            | Type of merge: `'inner'` (default), `'outer'`, `'left'`, `'right'`          |
| `on`             | Column(s) name(s) to join on (must be common in both DataFrames)            |
| `left_on`        | Column(s) from `left` DataFrame to join on                                  |
| `right_on`       | Column(s) from `right` DataFrame to join on                                 |
| `left_index`     | Use index from `left` as join key                                            |
| `right_index`    | Use index from `right` as join key                                           |


#### Output Control:

| Argument         | Description                                                                 |
|------------------|-----------------------------------------------------------------------------|
| `sort`           | Sort the result DataFrame by join keys (`True` or `False`, default `True`)  |
| `suffixes`       | Tuple of string suffixes to apply to overlapping columns (default: `('_x', '_y')`) |
| `copy`           | If `False`, avoid copying data unnecessarily (default is `True`)            |
| `indicator`      | If `True`, adds a column showing the source of each row in the merge        |
| `validate`       | Check if the merge is of a specific type (e.g., `'one_to_one'`, `'1:m'`)    |


## [ Merging on Index ]
- Normally, when you merge two DataFrames, you do it based on columns (e.g., `on="key"`). 
- But sometimes, you want to merge based on the row index. That's called merging on index


In [16]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [20]:
# "key" column from left1 and index of right1 to merge
pd.merge(left1, right1, left_on="key", right_index=True, sort=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [21]:
# since the default merge method is to intersect the join keys, we can instead form the union of them with an outer join
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [24]:
# with hierarchically indexed data, things are more complicated, as joining on index is equivalent to a multiple-key merge

lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                      "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})

righth_index = pd.MultiIndex.from_arrays([
    ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
    [2001, 2000, 2000, 2000, 2001, 2002]
    ])

righth = pd.DataFrame({
    "event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64", index=righth_index),
    "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64", index=righth_index)
})

print(lefth)
print(righth)

     key1  key2  data
0    Ohio  2000     0
1    Ohio  2001     1
2    Ohio  2002     2
3  Nevada  2001     3
4  Nevada  2002     4
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [26]:
# in this case, you have to indicate multiple columns to merge on as a list
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

# why?
    # the left DataFrame (lefth) uses two separate columns (key1 and key2) to define a logical key.
    # the right DataFrame (righth) uses a MultiIndex with two levels (e.g., "Ohio" + 2000).

    # To merge these two datasets:
        # we must align the two levels of the MultiIndex in righth with the two columns in lefth
        # That's exactly  what left_on=["key1", "key2"] does -- it tells pandas "Treat these two columns together as the key"

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [29]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True, how="outer")

,key1,key2,data,event1,event2
4,Nevada,2000,<NA>,2,3
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11


In [30]:
# using the indexes of both sides of the merge is also possible 

left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")

print(left2)
print(right2)

   Ohio  Nevada
a     1       2
c     3       4
e     5       6
   Missouri  Alabama
b         7        8
c         9       10
d        11       12
e        13       14


In [31]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [32]:
# DataFrame has a join instance method to simplify merging by index. It can also be used to combine many DataFrame objects having the same or similar indexes but nonoverlapping columns.
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [33]:
# compared with pandas.merge, DataFrame's join method performs a left join on the join keys by default. 
# it also supports joining the index of the passed DataFrame on one of the columns of the calling DataFrame
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [35]:
# lastly, for simple index-on-index merges, you can pass a list of DataFrames to join as an alternative to using the more general `pandas.concat` function
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"],
                       columns=["New York", "Oregon"])
print(another)

   New York  Oregon
a       7.0     8.0
c       9.0    10.0
e      11.0    12.0
f      16.0    17.0


In [36]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [37]:
left2.join([right2, another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0
